In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../src')

In [2]:
import torch
import torchvision

# Choose hardware acceleration if available
def choose_device() -> str:
    if torch.cuda.is_available():
        return "cuda:0"
    if hasattr(torch.backends, "mps"):
        if torch.backends.mps.is_available():
            return "mps"
    return "cpu"


device = torch.device(choose_device())
device

/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [3]:
# Adjust this path.
path_to_files = "../assets/imagenet"

# Load test data and make loaders.
x_batch = torch.load(f'{path_to_files}/x_batch.pt')
s_batch = torch.load(f'{path_to_files}/s_batch.pt')
x_batch, s_batch = x_batch.to(device), s_batch.to(device)
print(f"{len(x_batch)} matches found.")

17 matches found.


In [4]:
# Load pre-trained ResNet18 model.
model = torchvision.models.resnet18(pretrained=True)
model = model.to(device)
with torch.no_grad():
    model.eval()
    y_batch = model(x_batch).argmax(-1)
    y_batch = model(x_batch).argmax(-1)
    print('y_batch', y_batch)

/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


y_batch tensor([417, 308, 490, 474, 394, 490, 538, 538, 856, 857,  13, 308, 394, 856,
        470, 856,  13])


In [7]:
from captum.attr import IntegratedGradients

# Generate Integrated Gradients attributions of the first batch of the test set.
ig = IntegratedGradients(model)
a_batch, conv_delta = ig.attribute(x_batch, target=y_batch, baselines=torch.zeros_like(x_batch), internal_batch_size=64, return_convergence_delta=True)

In [17]:
from torchxai.metrics import completeness

# Compute completeness score.
completeness_scores = completeness(model, x_batch, a_batch, baselines=torch.zeros_like(x_batch), target=y_batch)

# check if the convergence delta and completeness are close, essentially they should be the same.
torch.allclose(completeness_scores.abs(), conv_delta.abs())

print(f"Completeness scores per example: {completeness_scores}")

Completeness scores per example: tensor([0.0626, 0.4576, 0.7652, 0.2514, 0.1281, 0.4750, 0.2360, 0.2140, 0.3280,
        0.0425, 0.6893, 0.3767, 0.3821, 0.4633, 0.0433, 0.2283, 0.2158],
       dtype=torch.float64)


In [18]:
# x = torch.arange(16).reshape(4,4).repeat_interleave(4, dim=0).repeat_interleave(4, dim=1).float()
def create_feature_masks(attributions: torch.Tensor, k:int = 16) -> torch.Tensor:
    feature_masks = []
    for attribution in attributions:
        dim_x, dim_y = attribution.shape[1] // k, attribution.shape[2] // k
        mask = torch.arange(dim_x*dim_y).view((dim_x, dim_y)).repeat_interleave(k, dim=0).repeat_interleave(k, dim=1).long()
        feature_masks.append(mask)
    return torch.stack(feature_masks).view_as(attributions)


In [21]:
from torchxai.metrics import monotonicity_corr_and_non_sens
x_batch, a_batch, y_batch = x_batch[:2], a_batch[:2], y_batch[:2]

# Only compute for the first two examples as this is very compute intensive for large input size of 1 x 224 x 224
# the channel dimension is perturbed together
a_batch_reduced = a_batch.sum(dim=1).unsqueeze(1)

# total number of computations for this include batch_size * 224 * 224 * 1 * 10 = batch_size * 501760 number of forward passes
# with max_features_processed_per_batch = 512, the number of forward passes is reduced to batch_size * 224 * 224 * 1 * 2 = batch_size * 100352
# to reduce the computations we can provide a feature mask to group the input features together
# the feature mask must be of the same shape as the attribution map
feature_mask = create_feature_masks(a_batch_reduced, k=16)

monotonicity_corr, non_sens_scores = monotonicity_corr_and_non_sens(model, x_batch, a_batch_reduced, feature_mask=feature_mask, target=y_batch, max_features_processed_per_batch=64)

print(f"Monotonicity correlation: {monotonicity_corr}")
print(f"Non-sensitivity scores: {non_sens_scores}")